# Hands On PySpark

Notebook provided un the <a href=https://www.coursera.org/projects/data-analysis-using-pyspark>Data Analysis Using Pyspark<a> Guided Project in Coursera.

# Task 1 :

Importing the modules 

In [10]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max 
import matplotlib.pyplot as plts

creating spark session

In [12]:
spark = SparkSession.builder.appName("spark_app").getOrCreate()
#sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

# Task 2 : 
importing the *Listenings.csv* file: 

let's check the data:

let's delete useless columns:

drop the null rows:

let's check the dataset again:

let's see the schema: 

let's see the shape of our dataframe: 

# Task 3:

**Query #0:**
select two columns: track and artist

**Query #1**:

Let's find all of the records of those users who have listened to ***Rihanna***

**Query #2:**

Let's find top 10 users who are fan of ***Rihanna***

**Query #3:**

find top 10 famous tracks 

**Query #4:**

find top 10 famous tracks of ***Rihanna*** 

**Query #5:**

find top 10 famous albums 

# Task 4 :
importing the ***genre.csv*** file:

let's check the data

Let's inner join these two data frames

**Query #6**

find top 10 users who are fan of ***pop*** music

**Query #7**

find top 10 famous genres

# Task 5:
**Query #8**

find out each user favourite genre

**Query #9**

find out how many pop,rock,metal and hip hop singers we have

and then visulize it using bar chart 

Now, let's visualize the results using ***matplotlib***

now lets visualize these two lists using a bar chart